In [1]:
import requests
import blankly
from blankly.data import PriceReader
import pandas as pd
from datetime import datetime

### their ideal backtest

In [2]:
def price_event(price, symbol, state: blankly.StrategyState):
    """ This function will give an updated price every 15 seconds from our definition below """
    state.variables['history'].append(price)
    rsi = blankly.indicators.rsi(state.variables['history'])
    curr_value = state.interface.account[state.base_asset].available
    if rsi[-1] < 30 and not curr_value:
        # Dollar cost average buy
        buy = blankly.trunc(state.interface.cash / price, 5)
        state.interface.market_order(symbol, side='buy', size=buy)
    elif rsi[-1] > 70 and curr_value:
        # Dollar cost average sell
        state.interface.market_order(symbol, side='sell', size=curr_value)


def init(symbol, state: blankly.StrategyState):
    # Download price data to give context to the algo
    state.variables['history'] = state.interface.history(symbol, to=150, return_as='deque',
                                                         resolution=state.resolution)['close']

In [3]:
if __name__ == "__main__":
    # This downloads an example CSV
    data = requests.get(
        'https://firebasestorage.googleapis.com/v0/b/blankly-6ada5.appspot.com/o/demo_data.csv?alt=media&token=acfa5c39-8f08-45dc-8be3-2033dc2b7b28').text
    with open('./price_examples.csv', 'w') as file:
        file.write(data)

    # Run on the keyless exchange, starting at 100k
    exchange = blankly.KeylessExchange(price_reader=PriceReader('./price_examples.csv', 'BTC-USD'))

    # Use our strategy helper
    strategy = blankly.Strategy(exchange)

    # Make the price event function above run every day
    strategy.add_price_event(price_event, symbol='BTC-USD', resolution='1d', init=init)

    # Backtest the strategy
    results = strategy.backtest(start_date=1598377600, end_date=1650067200, initial_values={'USD': 10000})
    print(results)


Backtesting...
Progress: [##########] 100% Done...

Historical Dataframes: 
Account History: 
     BTC           USD          time  Account Value (USD)
0    0.0  10000.000000  1.598378e+09         10000.000000
1    0.0  10000.000000  1.598378e+09         10000.000000
2    0.0  10000.000000  1.598464e+09         10000.000000
3    0.0  10000.000000  1.598550e+09         10000.000000
4    0.0  10000.000000  1.598637e+09         10000.000000
..   ...           ...           ...                  ...
595  0.0   9241.632621  1.649699e+09          9241.632621
596  0.0   9241.632621  1.649786e+09          9241.632621
597  0.0   9241.632621  1.649872e+09          9241.632621
598  0.0   9241.632621  1.649958e+09          9241.632621
599  0.0   9241.632621  1.650045e+09          9241.632621

[600 rows x 4 columns]
Account Returns: 
             time  value
0    1.598378e+09    NaN
1    1.598464e+09    0.0
2    1.598550e+09    0.0
3    1.598637e+09    0.0
4    1.598723e+09    0.0
..            ...

In [8]:
type(results)

blankly.exchanges.interfaces.paper_trade.backtest_result.BacktestResult

### data processing

In [109]:
aapl=pd.read_csv('price-data/AAPL.csv',index_col=0)

In [110]:
aapl

,open,high,low,close,volume
date,,,,,
2018-01-02,42.540001,43.075001,42.314999,43.064999,102223600
2018-01-03,43.132500,43.637501,42.990002,43.057499,118071600
2018-01-04,43.134998,43.367500,43.020000,43.257500,89738400
2018-01-05,43.360001,43.842499,43.262501,43.750000,94640000
2018-01-08,43.587502,43.902500,43.482498,43.587502,82271200
...,...,...,...,...,...
2023-06-23,185.550003,187.559998,185.009995,186.679993,53079300
2023-06-26,186.830002,188.050003,185.229996,185.270004,48088700
2023-06-27,185.889999,188.389999,185.669998,188.059998,50730800


I need to convert the date at epoch and make it a column named time

In [111]:
aapl.index=pd.to_datetime(aapl.index)
aapl['time'] = aapl.index.astype(int) // 10**9

# Reset the index to remove the datetime index
aapl.reset_index(drop=True, inplace=True)

# Move the 'time' column to the first position
cols = list(aapl.columns)
cols = [cols[-1]] + cols[:-1]
aapl = aapl[cols]

In [112]:
aapl

,time,open,high,low,close,volume
0,1514851200,42.540001,43.075001,42.314999,43.064999,102223600
1,1514937600,43.132500,43.637501,42.990002,43.057499,118071600
2,1515024000,43.134998,43.367500,43.020000,43.257500,89738400
3,1515110400,43.360001,43.842499,43.262501,43.750000,94640000
4,1515369600,43.587502,43.902500,43.482498,43.587502,82271200
...,...,...,...,...,...,...
1377,1687478400,185.550003,187.559998,185.009995,186.679993,53079300
1378,1687737600,186.830002,188.050003,185.229996,185.270004,48088700
1379,1687824000,185.889999,188.389999,185.669998,188.059998,50730800
1380,1687910400,187.929993,189.899994,187.600006,189.250000,51216800


In [113]:
aapl.to_csv('price-data/AAPL-USD.csv',index=False)

In [88]:
pred_price = pd.read_csv('/home/mehedi/Documents/blankly/price-data/apple_by_MA_etna.csv',index_col=0)

In [89]:
pred_price

,target
timestamp,
2018-04-11,42.520929
2018-04-12,42.724290
2018-04-13,42.968742
2018-04-14,43.228381
2018-04-15,43.340180
...,...
2023-06-25,186.499695
2023-06-26,186.573862
2023-06-27,186.828058


In [117]:
def price_event(price, symbol, state: blankly.StrategyState):
    """ This function will give an updated price every 15 seconds from our definition below """
    state.variables['history'].append(price)
    sma = blankly.indicators.sma(state.variables['history'])
    curr_value = state.interface.account[state.base_asset].available
    if price > sma[-1] and not curr_value:
        # Dollar cost average buy
        buy = blankly.trunc(state.interface.cash / price, 5)
        state.interface.market_order(symbol, side='buy', size=buy)
    elif price < sma[-1] and curr_value:
        # Dollar cost average sell
        state.interface.market_order(symbol, side='sell', size=curr_value)


def init(symbol, state: blankly.StrategyState):
    # Download price data to give context to the algo
    state.variables['history'] = state.interface.history(symbol, to=100, return_as='deque',
                                                         resolution=state.resolution)['close']

In [120]:
if __name__ == "__main__":
    

    # Run on the keyless exchange, starting at 100k
    exchange = blankly.KeylessExchange(price_reader=PriceReader('/home/mehedi/Documents/blankly/price-data/AAPL-USD.csv', 'AAPL-USD'))

    # Use our strategy helper
    strategy = blankly.Strategy(exchange)

    # Make the price event function above run every day
    strategy.add_price_event(price_event, symbol='AAPL-USD', resolution='1d', init=init)

    # Backtest the strategy
    results = strategy.backtest(start_date=1527120000, end_date=1687910400, initial_values={'USD': 1000000})
    print(results)


Backtesting...
Progress: [##########] 100% Done...

Historical Dataframes: 
Account History: 
             AAPL             USD          time  Account Value (USD)
0         0.00000  1000000.000000  1.527120e+09         1.000000e+06
1     21259.63396        0.000046  1.527120e+09         1.000000e+06
2     21259.63396        0.000046  1.527206e+09         1.002285e+06
3     21259.63396        0.000046  1.527293e+09         9.986713e+05
4     21259.63396        0.000046  1.527379e+09         9.986713e+05
...           ...             ...           ...                  ...
1859  18148.17604        0.000490  1.687651e+09         3.362313e+06
1860  18148.17604        0.000490  1.687738e+09         3.362313e+06
1861  18148.17604        0.000490  1.687824e+09         3.412946e+06
1862  18148.17604        0.000490  1.687910e+09         3.434542e+06
1863  18148.17604        0.000490  1.687997e+09         3.434542e+06

[1864 rows x 4 columns]
Account Returns: 
              time         value
0